In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux
numpy 1.16.3
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


# 2. Load Center Coords by Candidates

In [4]:
lungs = list(set(df_meta_test.index))
print('distinct lungs in testset:', len(lungs))

distinct lungs in testset: 222


In [5]:
list_tasks = []
size = settings.CUBE_SIZE
step = 32
start = [ size//2, size//2, size//2 ] # z,y,x

for uid in tqdm(lungs):
    meta = df_meta_test.loc[uid] # slice
    
    for z in range(start[0], int(meta.slice), step):
        for y in range(start[1], int(meta.height), step):
            for x in range(start[2], int(meta.width), step):
                task = {}
                task['seriesuid'] = uid
                task['vcoordX'] = x
                task['vcoordY'] = y
                task['vcoordZ'] = z
                
                # print(task)
                list_tasks.append(task)

100%|██████████| 222/222 [00:00<00:00, 248.56it/s]


In [6]:
df_tasks = pd.DataFrame(list_tasks, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
df_tasks = df_tasks.set_index('seriesuid')
df_tasks.index = df_tasks.index.astype('str')

print('total:', df_tasks.shape, 'lung:', len(set(df_tasks.index)))

if not os.path.exists(settings.SUBMISSION_DIR + 'candidates'):
    os.mkdir(settings.SUBMISSION_DIR + 'candidates')
    
df_tasks.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_cubes.csv', encoding='utf-8')


total: (208898, 3) lung: 222


# 3. Predict Results by tasks_cubes.csv

In [7]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

label = '5'
output_dir = './output/unet3d/1564380981/' # 5. 1564380981
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

W0729 12:02:47.293926 139989103707904 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 12:02:47.296077 139989103707904 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 12:02:47.303472 139989103707904 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [9]:
batch_size = 64
threshold_probability = 0.8
wtype = 'lung'
plot = False

In [10]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [11]:
list_results = []
print(wtype, 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_test.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 

    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='test', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred[:,:,:,0], itm)
                    if len(bboxes) > 0:
                        for i, box in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            if plot:
                                plot_cube((pred*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label +'.csv', encoding='utf-8')

    print('-'*100)
    

lung window
lung: 672396 tasks: 729 meta: 315.0 313.0 313.0
607/607 [==============================] - 22s 36ms/sample
total: (26, 6) lung: 1
----------------------------------------------------------------------------------------------------
lung: 656337 tasks: 1331 meta: 375.0 381.0 381.0
730/730 [==============================] - 12s 17ms/sample
total: (69, 6) lung: 2
----------------------------------------------------------------------------------------------------
lung: 647851 tasks: 900 meta: 315.0 350.0 350.0
581/581 [==============================] - 9s 16ms/sample
total: (102, 6) lung: 3
----------------------------------------------------------------------------------------------------
lung: 320473 tasks: 700 meta: 240.0 350.0 350.0
456/456 [==============================] - 8s 16ms/sample
total: (192, 6) lung: 4
----------------------------------------------------------------------------------------------------
lung: 656048 tasks: 900 meta: 300.0 350.0 350.0
521/521 [======

598/598 [==============================] - 10s 17ms/sample
total: (1338, 6) lung: 36
----------------------------------------------------------------------------------------------------
lung: 644447 tasks: 1296 meta: 300.0 400.0 400.0
685/685 [==============================] - 11s 16ms/sample
total: (1383, 6) lung: 37
----------------------------------------------------------------------------------------------------
lung: 688221 tasks: 567 meta: 255.0 311.0 311.0
354/354 [==============================] - 6s 18ms/sample
total: (1412, 6) lung: 38
----------------------------------------------------------------------------------------------------
lung: 633139 tasks: 1210 meta: 350.0 378.0 378.0
709/709 [==============================] - 12s 16ms/sample
total: (1479, 6) lung: 39
----------------------------------------------------------------------------------------------------
lung: 655422 tasks: 900 meta: 315.0 327.0 327.0
518/518 [==============================] - 8s 16ms/sample
total

444/444 [==============================] - 7s 16ms/sample
total: (4259, 6) lung: 106
----------------------------------------------------------------------------------------------------
lung: 688867 tasks: 1728 meta: 385.0 409.0 409.0
798/798 [==============================] - 13s 17ms/sample
total: (4292, 6) lung: 107
----------------------------------------------------------------------------------------------------
lung: 651646 tasks: 1000 meta: 325.0 350.0 350.0
681/681 [==============================] - 11s 16ms/sample
total: (4339, 6) lung: 108
----------------------------------------------------------------------------------------------------
lung: 663929 tasks: 968 meta: 265.0 373.0 373.0
515/515 [==============================] - 8s 16ms/sample
total: (4383, 6) lung: 109
----------------------------------------------------------------------------------------------------
lung: 656062 tasks: 1296 meta: 320.0 413.0 413.0
704/704 [==============================] - 11s 16ms/sample


401/401 [==============================] - 6s 16ms/sample
total: (5771, 6) lung: 141
----------------------------------------------------------------------------------------------------
lung: 672691 tasks: 900 meta: 300.0 322.0 322.0
673/673 [==============================] - 11s 16ms/sample
total: (5814, 6) lung: 142
----------------------------------------------------------------------------------------------------
lung: 630831 tasks: 1152 meta: 270.0 397.0 397.0
510/510 [==============================] - 8s 16ms/sample
total: (5840, 6) lung: 143
----------------------------------------------------------------------------------------------------
lung: 633133 tasks: 1440 meta: 340.0 391.0 391.0
648/648 [==============================] - 11s 16ms/sample
total: (5870, 6) lung: 144
----------------------------------------------------------------------------------------------------
lung: 656529 tasks: 900 meta: 315.0 324.0 324.0
549/549 [==============================] - 9s 16ms/sample
to

726/726 [==============================] - 11s 16ms/sample
total: (7068, 6) lung: 176
----------------------------------------------------------------------------------------------------
lung: 672957 tasks: 567 meta: 245.0 307.0 307.0
405/405 [==============================] - 6s 16ms/sample
total: (7092, 6) lung: 177
----------------------------------------------------------------------------------------------------
lung: 373375 tasks: 729 meta: 300.0 289.0 289.0
501/501 [==============================] - 8s 16ms/sample
total: (7125, 6) lung: 178
----------------------------------------------------------------------------------------------------
lung: 688160 tasks: 800 meta: 265.0 331.0 331.0
510/510 [==============================] - 8s 16ms/sample
total: (7141, 6) lung: 179
----------------------------------------------------------------------------------------------------
lung: 660392 tasks: 900 meta: 290.0 331.0 331.0
587/587 [==============================] - 9s 16ms/sample
total

540/540 [==============================] - 8s 16ms/sample
total: (8507, 6) lung: 211
----------------------------------------------------------------------------------------------------
lung: 689303 tasks: 1440 meta: 327.0 403.0 403.0
520/520 [==============================] - 8s 16ms/sample
total: (8524, 6) lung: 212
----------------------------------------------------------------------------------------------------
lung: 656748 tasks: 1089 meta: 300.0 373.0 373.0
578/578 [==============================] - 9s 16ms/sample
total: (8554, 6) lung: 213
----------------------------------------------------------------------------------------------------
lung: 678534 tasks: 900 meta: 300.0 351.0 351.0
647/647 [==============================] - 10s 16ms/sample
total: (8595, 6) lung: 214
----------------------------------------------------------------------------------------------------
lung: 647539 tasks: 1089 meta: 310.0 368.0 368.0
684/684 [==============================] - 11s 16ms/sample
t

In [14]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label +'.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))



results: (8852, 6) distinct lung: 222


In [15]:
filepath_1 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_1.csv'
filepath_5 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_5.csv'
if os.path.exists(filepath_1) and os.path.exists(filepath_5):
    df_lung_1 = pd.read_csv(filepath_or_buffer=filepath_1, index_col=['seriesuid'])
    df_lung_1.index = df_lung_1.index.astype('str')
    print('lung window 1:', df_lung_1.shape)
    
    df_lung_5 = pd.read_csv(filepath_or_buffer=filepath_5, index_col=['seriesuid'])
    df_lung_5.index = df_lung_5.index.astype('str')
    print('lung window 5:', df_lung_5.shape)
    
    df_lung = pd.concat([df_lung_1, df_lung_5])
    print('lung windows merged:', df_lung.shape)
    df_lung.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'.csv', encoding='utf-8')

lung window 1: (4172, 6)
lung window 5: (8852, 6)
lung windows merged: (13024, 6)
